# Create Slice for Fabric Rack
This is the slice creation for the ```RTT_Window_Size``` tutorial, This is the first step of the assignment, to create the slice on fabric using fablib
 

## 1. Set up the Experiment

In this section you will use the Fablib maager to create a new slice that is composed of 2 nodes for the Window Size Assignment.
### 1.1 Reserve Resources
In this section we will declare the Structure of the Slice we will be using.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "RttWindowTCP"
site = "EDUKY"
print(f'site = {site}')

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

try:
    # Create a slice
    slice = fablib.new_slice(name=slice_name)
    #Server
    Server = slice.add_node(name="Server", site=site)
    Server.set_capacities(cores=cores, ram=ram, disk=disk)
    Server.set_image(image)
    S = Server.add_component(model=nicmodel, name="Server_P").get_interfaces()[0] 
    
    
    # Client
    Client = slice.add_node(name="Client", site=site)
    Client.set_capacities(cores=cores, ram=ram, disk=disk)
    Client.set_image(image)
    C = Client.add_component(model = nicmodel, name="Client_P").get_interfaces()[0] 

    LAN_CONN = slice.add_l2network(name="LAN_CONN", interfaces=[S, C])
    
    # Submit the slice
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

### 1.2 Set up the Experiment Network
This section of the slice creation, once the nodes are active, will asign the network spaces some predefined values that we will use for the Assgnment

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("10.1.1.0/24")
    available_ips = list(subnet)[1:]

    network_name = 'LAN_CONN'

    for node in slice.get_nodes():
        node_iface = node.get_interface(network_name=network_name)  
        node_addr = available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_addr, subnet=subnet)
        
        print(f'### Ip adress for node {node.get_name()} is {node_addr} ###')
        
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
        if (node.get_name() == "Client"):
            A_Device_Name = node_iface.get_device_name()
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")

except Exception as e:
    print(f"Fail: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
This section of the slice creation, we will Add any aditional software, tools or scripts that we need for our experiments.

In [ ]:
# Install Software
try:
    INTERFACE_NAME_HOLDER = "INTERFACE_NAME"

    for node in slice.get_nodes():        
        node.execute("sudo apt update")
        node.execute("sudo apt install net-tools")
        node.execute("sudo apt install iperf")

        #Graphics
        node.execute("wget https://github.com/fabric-testbed/teaching-materials/raw/main/assignments/RTT%20and%20Window%20Size%20on%20TCP/scripts/RT-data-analysis.R")
        node.execute("wget https://github.com/fabric-testbed/teaching-materials/raw/main/assignments/RTT%20and%20Window%20Size%20on%20TCP/scripts/RT-output.sh")
        node.execute(f"sed -i 's/{INTERFACE_NAME_HOLDER}/{A_Device_Name}/' RT-output.sh")
        node.execute("sudo apt-get -y install moreutils r-base-core r-cran-ggplot2 r-cran-littler;sudo sysctl -w net.ipv4.tcp_no_metrics_save=1;chmod +x RT-output.sh ")

except Exception as e:
    print(f"Fail: {e}")

## Continue to The experiment notebook

Once You have completed this notebook you should be able to continue to the RTT Window size Notebook, you can ether open it on the explorer or click [Here](./RTT_Window_Size.ipynb) to open the next notebook.